In [1]:
import redis

In [95]:
r = redis.StrictRedis(host='localhost', port=6379, db=0, charset="utf-8", decode_responses=True)

In [3]:
import pandas as pd

https://www.kaggle.com/ritresearch/happydb

In [5]:
data = pd.read_csv('cleaned_hm.csv')

In [6]:
data.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection
2,27675,1936,24h,I went to the gym this morning and did yoga.,I went to the gym this morning and did yoga.,True,1,NaN,exercise
3,27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
4,27677,6227,24h,I went with grandchildren to butterfly display...,I went with grandchildren to butterfly display...,True,1,NaN,affection


In [92]:
texts = data.set_index('wid')['cleaned_hm'].to_dict()

In [96]:
r.hmset('texts', texts)

True

In [25]:
pdict = dict()
for i in data['predicted_category'].unique().tolist():
    data_slice = data[data['predicted_category'] == i]
    pdict[i] = data_slice['cleaned_hm'].tolist()

In [98]:
pdict = data.set_index('predicted_category')['cleaned_hm'].to_dict()

In [126]:
r.hmset('categories', pdict)

True

In [127]:
r.hgetall('categories')

{'achievement': 'Those rare, perfectly timed naps that leave you feeling refreshed and not groggy.\r\nWaking up before your alarm goes off.\r\nBeing in a great mood for no reason at all.',
 'enjoy_the_moment': 'Had a can of Pepsi to drink.',
 'affection': 'Cuddling with my girlfriend last night.',
 'nature': 'I was able to weed my entire garden and plant tomatoes and basil before it rained.',
 'exercise': 'I had a great workout last night.',
 'bonding': "I had a great meeting yesterday at work with my boss and a few colleagues and we went out for lunch afterward; everybody was excited by the projects we're working on and how efficient our team is.",
 'leisure': 'Finally got to watch the new Resident Evil movie.'}

#### Функции

In [156]:
def getting_text(k):
    for name in r.keys():
        text = r.hget(name, k)
        if text is not None:
            return text
        else:
            return print('No such text')

In [138]:
print(getting_text('3977'))

I lifted my heaviest weights that I have ever lifted before during exercise.


In [139]:
getting_text('affection')

'Cuddling with my girlfriend last night.'

In [145]:
def adding(where, k, v):
    if r.exists(where) is 1:
        r.hset(where, k, v)
    else:
        print('Nope. Possible hashes: ', r.keys())
    return

In [143]:
adding('texts', '100500', 'I was happy to win the game')

In [146]:
adding('age', '10', 'Florida')

Nope. Possible hashes:  ['categories', 'texts', 'HappyDB']


In [147]:
getting_text('100500')

I was happy to win the game


In [148]:
def deleting(k):
    for name in r.keys():
        if r.hdel(name, k) != 0:
            print('Done')
    return

In [149]:
deleting('100500')

Done


In [158]:
getting_text('100500')

No such text


In [203]:
def editing(name):
    if r.exists(name) == 1:
        print(list(r.hgetall(name).items()))
        ke = input('Type the key you want to change: ')
        new = input('Type the new value: ')
        r.hmset(name, {ke:new})
        return 'Done'
    else:
        return 'no such hash'

In [204]:
editing('categories')

[('achievement', 'Those rare, perfectly timed naps that leave you feeling refreshed and not groggy.\r\nWaking up before your alarm goes off.\r\nBeing in a great mood for no reason at all.'), ('enjoy_the_moment', 'Had a can of Pepsi to drink.'), ('affection', 'Cuddling with my girlfriend last night.'), ('nature', 'I was able to weed my entire garden and plant tomatoes and basil before it rained.'), ('exercise', 'I had a great workout last night.'), ('bonding', "I had a great meeting yesterday at work with my boss and a few colleagues and we went out for lunch afterward; everybody was excited by the projects we're working on and how efficient our team is."), ('leisure', 'I')]
Type the key you want to change: leisure
Type the new value: I was walking in the night


'Done'

In [205]:
r.hgetall('categories')

{'achievement': 'Those rare, perfectly timed naps that leave you feeling refreshed and not groggy.\r\nWaking up before your alarm goes off.\r\nBeing in a great mood for no reason at all.',
 'enjoy_the_moment': 'Had a can of Pepsi to drink.',
 'affection': 'Cuddling with my girlfriend last night.',
 'nature': 'I was able to weed my entire garden and plant tomatoes and basil before it rained.',
 'exercise': 'I had a great workout last night.',
 'bonding': "I had a great meeting yesterday at work with my boss and a few colleagues and we went out for lunch afterward; everybody was excited by the projects we're working on and how efficient our team is.",
 'leisure': 'I was walking in the night'}

In [172]:
print(list(r.hgetall('categories').keys()))

['achievement', 'enjoy_the_moment', 'affection', 'nature', 'exercise', 'bonding', 'leisure']


In [218]:
r.keys()

['categories', 'texts', 'HappyDB']

In [213]:
r.expire('HappyDB', 100)

True

In [216]:
r.ttl('HappyDB')

53

In [220]:
r.keys()

['categories', 'texts']